# Inputs

In [3]:
import pandas as pd
import os
import re
import csv
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile
import numpy as np
import matplotlib.patches as mpatches

# Import data

## Transformations (merges and acquisitions)

In [30]:
path_transformations = r"C:\Users\joaof\OneDrive\Education\University\PhD_UVM\Courses\202526\Fall\CS5870A_DataScienceI\Project\DSE-Project\Data_Processing\inputs\FED_transformations\CSV_TRANSFORMATIONS.CSV"
df_transform = pd.read_csv(path_transformations)
df_transform.head()

,#ID_RSSD_PREDECESSOR,ID_RSSD_SUCCESSOR,D_DT_TRANS,TRNSFM_CD,ACCT_METHOD,DT_TRANS
0,28,75026,01/01/1994 00:00:00,1,0,19940101
1,46,1003848,10/01/1993 00:00:00,1,0,19931001
2,55,951951,01/11/1982 00:00:00,1,0,19820111
3,64,1213683,07/30/1988 00:00:00,50,0,19880730
4,82,41393,01/01/2000 00:00:00,1,0,20000101


In [31]:
df_transform['ACCT_METHOD'].value_counts()

ACCT_METHOD
0    48338
1     5933
2     4473
Name: count, dtype: int64

In [32]:
df_transform['TRNSFM_CD'].value_counts()

TRNSFM_CD
1     54325
50     4223
7        85
9        78
5        33
Name: count, dtype: int64

In [33]:
df_transform.groupby(['#ID_RSSD_PREDECESSOR','ID_RSSD_SUCCESSOR']).size().sort_values()

#ID_RSSD_PREDECESSOR  ID_RSSD_SUCCESSOR
28                    75026                1
46                    1003848              1
55                    951951               1
64                    1213683              1
82                    41393                1
                                          ..
954550                3132667              2
887126                2353595              2
2581558               836852               2
992057                139656               2
3074664               3080535              2
Length: 58736, dtype: int64

'''Some institutions in relationships are not in financial data DB, because:
1) institutiosn have 45 to submit information for the quarter
2) institutions are not required to fill specific formularies because of their size'''

In [39]:
df_transform[(df_transform['#ID_RSSD_PREDECESSOR'].astype(str)=='1020854') | (df_transform['ID_RSSD_SUCCESSOR'].astype(str)=='1020854')]

,#ID_RSSD_PREDECESSOR,ID_RSSD_SUCCESSOR,D_DT_TRANS,TRNSFM_CD,ACCT_METHOD,DT_TRANS
37884,1060430,1020854,08/01/1992 00:00:00,1,0,19920801
37941,1062340,1020854,08/01/1992 00:00:00,1,0,19920801


## Relationships (ownership relationships)

In [21]:
path_relationships_ = r"C:\Users\joaof\OneDrive\Education\University\PhD_UVM\Courses\202526\Fall\CS5870A_DataScienceI\Project\DSE-Project\Data_Processing\inputs\FED_relationships\CSV_RELATIONSHIPS.CSV"

In [22]:
df_relationships = pd.read_csv(path_relationships_)
df_relationships.head()

,#ID_RSSD_PARENT,ID_RSSD_OFFSPRING,D_DT_START,RELN_LVL,CTRL_IND,D_DT_RELN_EST,EQUITY_IND,OTHER_BASIS_IND,PCT_EQUITY,PCT_OTHER,...,REG_IND,PCT_EQUITY_BRACKET,PCT_EQUITY_FORMAT,D_DT_END,MB_COST,FC_IND,REGK_INV,DT_END,DT_RELN_EST,DT_START
0,130,1081305,06/27/1984 00:00:00,1,1,06/27/1984 00:00:00,2,0,80.0,0.0,...,1,80-100,BRACKET,02/14/1986 00:00:00,0,0,0,19860214,19840627,19840627
1,279,3923155,01/23/2008 00:00:00,1,1,01/23/2008 00:00:00,2,0,100.0,0.0,...,1,100,BRACKET,12/31/9999 00:00:00,0,0,0,99991231,20080123,20080123
2,505,2913168,05/27/1999 00:00:00,1,1,05/27/1999 00:00:00,2,0,80.0,0.0,...,1,80-100,BRACKET,06/29/2007 00:00:00,0,0,0,20070629,19990527,19990527
3,505,2913168,06/30/2007 00:00:00,1,1,05/27/1999 00:00:00,2,0,100.0,0.0,...,1,100,BRACKET,12/31/9999 00:00:00,0,0,0,99991231,19990527,20070630
4,505,3079566,06/01/2000 00:00:00,1,1,06/01/2000 00:00:00,2,0,80.0,0.0,...,1,80-100,BRACKET,02/12/2003 00:00:00,0,0,0,20030212,20000601,20000601


In [74]:
df_relationships[df_relationships['ID_RSSD_OFFSPRING'].astype(str)=='1021570']

,#ID_RSSD_PARENT,ID_RSSD_OFFSPRING,D_DT_START,RELN_LVL,CTRL_IND,D_DT_RELN_EST,EQUITY_IND,OTHER_BASIS_IND,PCT_EQUITY,PCT_OTHER,...,REG_IND,PCT_EQUITY_BRACKET,PCT_EQUITY_FORMAT,D_DT_END,MB_COST,FC_IND,REGK_INV,DT_END,DT_RELN_EST,DT_START
101144,1199808,1021570,07/03/1980 00:00:00,1,2,07/03/1980 00:00:00,1,0,20.0,0.0,...,1,<25,EXACT,12/30/1981 00:00:00,0,0,0,19811230,19800703,19800703
101145,1199808,1021570,12/31/1981 00:00:00,1,1,07/03/1980 00:00:00,1,0,26.0,0.0,...,1,25-50,EXACT,12/30/1982 00:00:00,0,0,0,19821230,19800703,19811231
101146,1199808,1021570,12/31/1982 00:00:00,1,1,07/03/1980 00:00:00,1,0,34.0,0.0,...,1,25-50,EXACT,12/30/1984 00:00:00,0,0,0,19841230,19800703,19821231
101147,1199808,1021570,12/31/1984 00:00:00,1,1,07/03/1980 00:00:00,1,0,39.0,0.0,...,1,25-50,EXACT,12/30/1992 00:00:00,0,0,0,19921230,19800703,19841231
101148,1199808,1021570,12/31/1992 00:00:00,1,1,07/03/1980 00:00:00,1,0,29.0,0.0,...,1,25-50,EXACT,07/14/1997 00:00:00,0,0,0,19970714,19800703,19921231
101149,1199808,1021570,07/15/1997 00:00:00,1,1,07/03/1980 00:00:00,1,0,30.0,0.0,...,1,25-50,EXACT,07/19/1998 00:00:00,0,0,0,19980719,19800703,19970715
101150,1199808,1021570,07/20/1998 00:00:00,1,1,07/03/1980 00:00:00,1,0,29.0,0.0,...,1,25-50,EXACT,05/30/2000 00:00:00,0,0,0,20000530,19800703,19980720
101151,1199808,1021570,05/31/2000 00:00:00,1,1,07/03/1980 00:00:00,1,0,30.0,0.0,...,1,25-50,EXACT,12/26/2001 00:00:00,0,0,0,20011226,19800703,20000531
193772,3115268,1021570,12/26/2001 00:00:00,1,1,12/26/2001 00:00:00,0,2,0.0,100.0,...,1,0,N/A,04/30/2003 00:00:00,0,0,0,20030430,20011226,20011226


In [76]:
df_relationships[df_relationships['#ID_RSSD_PARENT'].astype(str)=='1021570']

,#ID_RSSD_PARENT,ID_RSSD_OFFSPRING,D_DT_START,RELN_LVL,CTRL_IND,D_DT_RELN_EST,EQUITY_IND,OTHER_BASIS_IND,PCT_EQUITY,PCT_OTHER,...,REG_IND,PCT_EQUITY_BRACKET,PCT_EQUITY_FORMAT,D_DT_END,MB_COST,FC_IND,REGK_INV,DT_END,DT_RELN_EST,DT_START
43792,1021570,747743,03/02/1986 00:00:00,1,1,03/02/1986 00:00:00,1,0,94.0,0.0,...,1,80-100,EXACT,12/27/2001 00:00:00,0,0,0,20011227,19860302,19860302
43793,1021570,747743,12/28/2001 00:00:00,1,1,03/02/1986 00:00:00,1,0,99.0,0.0,...,1,80-100,EXACT,11/09/2005 00:00:00,0,0,0,20051109,19860302,20011228
43794,1021570,747743,11/10/2005 00:00:00,1,1,03/02/1986 00:00:00,1,0,100.0,0.0,...,1,100,EXACT,12/31/2017 00:00:00,0,0,0,20171231,19860302,20051110
43795,1021570,810544,10/10/1978 00:00:00,1,1,10/10/1978 00:00:00,1,0,82.0,0.0,...,1,80-100,EXACT,01/30/1996 00:00:00,0,0,0,19960130,19781010,19781010
43796,1021570,810544,01/31/1996 00:00:00,1,1,10/10/1978 00:00:00,1,0,87.0,0.0,...,1,80-100,EXACT,06/01/1998 00:00:00,0,0,0,19980601,19781010,19960131
43797,1021570,810544,06/02/1998 00:00:00,1,1,10/10/1978 00:00:00,1,0,88.0,0.0,...,1,80-100,EXACT,12/27/2001 00:00:00,0,0,0,20011227,19781010,19980602
43798,1021570,810544,12/28/2001 00:00:00,1,1,10/10/1978 00:00:00,1,0,91.0,0.0,...,1,80-100,EXACT,03/17/2002 00:00:00,0,0,0,20020317,19781010,20011228
43799,1021570,810544,03/18/2002 00:00:00,1,1,10/10/1978 00:00:00,1,0,100.0,0.0,...,1,100,EXACT,12/31/9999 00:00:00,0,0,0,99991231,19781010,20020318
43800,1021570,1199808,07/03/1980 00:00:00,1,1,07/03/1980 00:00:00,1,0,36.0,0.0,...,1,25-50,EXACT,12/31/1992 00:00:00,0,0,0,19921231,19800703,19800703
43801,1021570,1199808,01/01/1993 00:00:00,1,1,07/03/1980 00:00:00,1,0,37.0,0.0,...,1,25-50,EXACT,01/30/1995 00:00:00,0,0,0,19950130,19800703,19930101


In [28]:
print(df_relationships.iloc[0])
print(df_relationships.columns)

#ID_RSSD_PARENT                       130
ID_RSSD_OFFSPRING                 1081305
D_DT_START            06/27/1984 00:00:00
RELN_LVL                                1
CTRL_IND                                1
D_DT_RELN_EST         06/27/1984 00:00:00
EQUITY_IND                              2
OTHER_BASIS_IND                         0
PCT_EQUITY                           80.0
PCT_OTHER                             0.0
REASON_ROW_CRTD                         1
REASON_TERM_RELN                        1
REG_IND                                 1
PCT_EQUITY_BRACKET               80-100  
PCT_EQUITY_FORMAT                BRACKET 
D_DT_END              02/14/1986 00:00:00
MB_COST                                 0
FC_IND                                  0
REGK_INV                                0
DT_END                           19860214
DT_RELN_EST                      19840627
DT_START                         19840627
Name: 0, dtype: object
Index(['#ID_RSSD_PARENT', 'ID_RSSD_OFFSPRING', 'D_DT_

In [26]:
df_relationships['RELN_LVL'].value_counts()

RELN_LVL
1    283116
4      1063
2       642
3        36
Name: count, dtype: int64

In [27]:
df_relationships['CTRL_IND'].value_counts()

CTRL_IND
1    256691
2     28034
0       132
Name: count, dtype: int64

In [29]:
df_relationships['EQUITY_IND'].value_counts()

EQUITY_IND
2    181192
1     53412
0     50253
Name: count, dtype: int64

**Questions:**
- In DB, branches (companies that detained by others) appear individually in the DB?
    - If that is the case, should we consolidate them all?

## Branches - active

In [48]:
path_active = r"C:\Users\joaof\OneDrive\Education\University\PhD_UVM\Courses\202526\Fall\CS5870A_DataScienceI\Project\DSE-Project\Data_Processing\inputs\FED_attributes_active\CSV_ATTRIBUTES_ACTIVE.CSV"

In [49]:
path_branches = r"C:\Users\joaof\OneDrive\Education\University\PhD_UVM\Courses\202526\Fall\CS5870A_DataScienceI\Project\DSE-Project\Data_Processing\inputs\FED_attributes_branches\CSV_ATTRIBUTES_BRANCHES.CSV"

In [50]:
path_closed = r"C:\Users\joaof\OneDrive\Education\University\PhD_UVM\Courses\202526\Fall\CS5870A_DataScienceI\Project\DSE-Project\Data_Processing\inputs\FED_attributes_closed\CSV_ATTRIBUTES_CLOSED.CSV"

In [55]:
df_active = pd.read_csv(path_active)
df_active.head()

C:\Users\joaof\AppData\Local\Temp\ipykernel_25016\4125359545.py:1: DtypeWarning: Columns (35,45,47,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df_active = pd.read_csv(path_active)


,#ID_RSSD,D_DT_START,D_DT_END,BHC_IND,BROAD_REG_CD,CHTR_AUTH_CD,CHTR_TYPE_CD,FBO_4C9_IND,FHC_IND,FUNC_REG,...,URL,SLHC_IND,SLHC_TYPE_IND,PRIM_FED_REG,STATE_INC_CD,CNTRY_INC_CD,STATE_INC_ABBR_NM,CNTRY_INC_NM,ID_LEI,IHC_IND
0,37,04/15/2009 00:00:00,12/31/9999 00:00:00,0,1,2,200,0,0,0,...,0,0,0,FDIC,0,0,0,NaN,0,0
1,73,12/31/2008 00:00:00,12/31/9999 00:00:00,0,2,1,330,0,0,0,...,0,0,0,NCUA,0,0,0,NaN,0,0
2,242,01/01/2012 00:00:00,12/31/9999 00:00:00,0,1,2,200,0,0,0,...,WWW.FCBXENIAFLORA.COM/INDEX.HTML,0,0,FRS,17,0,IL,NaN,0,0
3,279,12/04/2023 00:00:00,12/31/9999 00:00:00,0,1,2,300,0,0,0,...,0,0,0,FDIC,48,0,TX,NaN,54930067OMWEWTHVDO95,0
4,354,07/01/2021 00:00:00,12/31/9999 00:00:00,0,1,2,200,0,0,0,...,0,0,0,FDIC,0,0,0,NaN,549300T61O0X0RJMQ375,0


In [56]:
df_branches = pd.read_csv(path_branches)
df_branches.head()

C:\Users\joaof\AppData\Local\Temp\ipykernel_25016\3734476464.py:1: DtypeWarning: Columns (20,45,63,64,70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df_branches = pd.read_csv(path_branches)


,#ID_RSSD,D_DT_START,D_DT_END,BHC_IND,BROAD_REG_CD,CHTR_AUTH_CD,CHTR_TYPE_CD,FBO_4C9_IND,FHC_IND,FUNC_REG,...,URL,SLHC_IND,SLHC_TYPE_IND,PRIM_FED_REG,STATE_INC_CD,CNTRY_INC_CD,STATE_INC_ABBR_NM,CNTRY_INC_NM,ID_LEI,IHC_IND
0,726636,01/15/2000 00:00:00,11/09/2012 00:00:00,0,0,0,0,0,0,0,...,0,0,0,NaN,0,0,0,NaN,0,0
1,728742,08/21/2018 00:00:00,12/31/9999 00:00:00,0,0,0,0,0,0,0,...,0,0,0,NaN,0,0,0,NaN,0,0
2,5785188,09/06/2022 00:00:00,12/31/9999 00:00:00,0,0,0,0,0,0,0,...,0,0,0,NaN,0,0,0,NaN,0,0
3,3459908,12/31/1959 00:00:00,12/31/9999 00:00:00,0,0,0,0,0,0,0,...,0,0,0,NaN,0,0,0,NaN,0,0
4,3840083,03/01/2025 00:00:00,12/31/9999 00:00:00,0,0,0,0,0,0,0,...,0,0,0,NaN,0,0,0,NaN,0,0


In [57]:
df_closed = pd.read_csv(path_closed)
df_closed.head()

C:\Users\joaof\AppData\Local\Temp\ipykernel_25016\4161748934.py:1: DtypeWarning: Columns (35,45,63,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df_closed = pd.read_csv(path_closed)


,#ID_RSSD,D_DT_START,D_DT_END,BHC_IND,BROAD_REG_CD,CHTR_AUTH_CD,CHTR_TYPE_CD,FBO_4C9_IND,FHC_IND,FUNC_REG,...,URL,SLHC_IND,SLHC_TYPE_IND,PRIM_FED_REG,STATE_INC_CD,CNTRY_INC_CD,STATE_INC_ABBR_NM,CNTRY_INC_NM,ID_LEI,IHC_IND
0,28451,12/31/1959 00:00:00,10/15/1985 00:00:00,0,1,2,200,0,0,0,...,0,0,0,FDIC,0,0,0,NaN,0,0
1,28479,06/30/2003 00:00:00,12/31/2007 00:00:00,0,2,1,330,0,0,0,...,0,0,0,NCUA,0,0,0,NaN,0,0
2,28497,01/02/1998 00:00:00,07/31/2004 00:00:00,0,2,2,330,0,0,0,...,0,0,0,NCUA,0,0,0,NaN,0,0
3,28572,01/01/2012 00:00:00,05/23/2014 00:00:00,0,1,2,300,0,0,0,...,0,0,0,FDIC,39,0,OH,NaN,0,0
4,28590,01/30/2006 00:00:00,09/30/2007 00:00:00,0,2,2,330,0,0,0,...,0,0,0,NCUA,0,0,0,NaN,0,0


In [71]:
df_relationships[(df_relationships['CTRL_IND']==1) & (df_relationships['#ID_RSSD_PARENT'].astype(str) == '2589723')]

,#ID_RSSD_PARENT,ID_RSSD_OFFSPRING,D_DT_START,RELN_LVL,CTRL_IND,D_DT_RELN_EST,EQUITY_IND,OTHER_BASIS_IND,PCT_EQUITY,PCT_OTHER,...,REG_IND,PCT_EQUITY_BRACKET,PCT_EQUITY_FORMAT,D_DT_END,MB_COST,FC_IND,REGK_INV,DT_END,DT_RELN_EST,DT_START
164633,2589723,459671,10/15/1997 00:00:00,1,1,10/15/1997 00:00:00,1,0,100.0,0.0,...,1,100,EXACT,12/31/9999 00:00:00,0,0,0,99991231,19971015,19971015
164634,2589723,849777,11/01/2018 00:00:00,1,1,11/01/2018 00:00:00,1,0,100.0,0.0,...,1,100,EXACT,03/31/2019 00:00:00,0,0,0,20190331,20181101,20181101


In [72]:
df_active[df_active['#ID_RSSD'].astype(str)=='1021570'].iloc[0].to_dict()

{'#ID_RSSD': 1021570,
 'D_DT_START': '01/01/2018 00:00:00',
 'D_DT_END': '12/31/9999 00:00:00',
 'BHC_IND': 1,
 'BROAD_REG_CD': 3,
 'CHTR_AUTH_CD': 0,
 'CHTR_TYPE_CD': 500,
 'FBO_4C9_IND': 0,
 'FHC_IND': 0,
 'FUNC_REG': 0,
 'INSUR_PRI_CD': 0,
 'MBR_FHLBS_IND': 0,
 'MBR_FRS_IND': 0,
 'SEC_RPTG_STATUS': 2,
 'EST_TYPE_CD': 1,
 'BANK_CNT': 1.0,
 'BNK_TYPE_ANALYS_CD': 0,
 'D_DT_EXIST_CMNC': '10/10/1978 00:00:00',
 'D_DT_EXIST_TERM': '12/31/9999 00:00:00',
 'FISC_YREND_MMDD': 1231,
 'D_DT_INSUR': nan,
 'D_DT_OPEN': '10/10/1978 00:00:00',
 'FNCL_SUB_HOLDER': 0,
 'FNCL_SUB_IND': 0,
 'IBF_IND': 0,
 'ID_RSSD_HD_OFF': 0,
 'MJR_OWN_MNRTY': 0,
 'NM_LGL': 'VIKING CORPORATION, THE                                                                                                 ',
 'NM_SHORT': 'VIKING CORP                   ',
 'NM_SRCH_CD': 4720467,
 'ORG_TYPE_CD': 1,
 'REASON_TERM_CD': 0,
 'CNSRVTR_CD': 0,
 'ENTITY_TYPE': 'BHC',
 'AUTH_REG_DIST_FRS': 10,
 'ACT_PRIM_CD': 551111,
 'CITY': 'OMAHA',
 'CNT

In [61]:
df_branches[df_branches['#ID_RSSD'].astype(str)=='2589723']

,#ID_RSSD,D_DT_START,D_DT_END,BHC_IND,BROAD_REG_CD,CHTR_AUTH_CD,CHTR_TYPE_CD,FBO_4C9_IND,FHC_IND,FUNC_REG,...,URL,SLHC_IND,SLHC_TYPE_IND,PRIM_FED_REG,STATE_INC_CD,CNTRY_INC_CD,STATE_INC_ABBR_NM,CNTRY_INC_NM,ID_LEI,IHC_IND
